<a href="https://colab.research.google.com/github/soyoonjeong/Running_Machine/blob/main/%ED%97%AC%EC%8A%A4%EC%9E%A5_%EC%9C%84%EB%8F%84_%EA%B2%BD%EB%8F%84_%EB%B3%80%ED%99%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
import re
import time
from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotVisibleException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu

In [ ]:
# 웹 드라이버 로딩
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome("chromedriver", chrome_options=chrome_options)
driver.implicitly_wait(3) # driver가 로딩될 때까지 기다리기 위해 implicitly_wait(3) 줌

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


# 헬스장 url 추출 

In [ ]:
region = '동작구'

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/datasets/헬스장 크롤링/{0}/company.csv".format(region))
df.head()

,id,name,img,grade,lat,lng,phoneNumber,address
0,0,헤라클레스,https://search.pstatic.net/common/?autoRotate=...,NaN,37.505922,126.952942,824-8688,서울특별시 동작구 흑석로6가길 21 (흑석동)
1,1,명문헬스클럽,https://search.pstatic.net/common/?autoRotate=...,NaN,37.493530,126.954541,814-0579,"서울특별시 동작구 상도로 399, 202호 (상도동, 삼호상가)"
2,2,에브리데이짐,https://search.pstatic.net/common/?autoRotate=...,4.95,37.504734,126.935446,817-6400,서울특별시 동작구 상도로 179 (상도동)
3,3,국제 헬스,NaN,NaN,37.511802,126.932055,02-814-4155,서울특별시 동작구 등용로4길 148 (노량진동)
4,4,엔짐,https://search.pstatic.net/common/?autoRotate=...,4.78,37.497142,126.941727,817-8990,"서울특별시 동작구 양녕로26길 27, 5층 (상도동, 삼익아파트)"


In [ ]:
df['naver_map_url'] = "none"

In [ ]:
for i in range(len(df)):
  keyword = df['name'][i]
  print("이번에 찾을 키워드 : ", keyword)
  try:
    if df['naver_map_url'][i]!='none': # 이미 url 있는 거 다 패스 
      continue
    address_idx_start = df['address'][i].index("(")+1
    address_idx_end =  len(df['address'][i])-df['address'][i][::-1].index("동") # 동 이름 추출 
    naver_map_search_url = "https://map.naver.com/v5/search/"+region+" "+df['address'][i][address_idx_start:address_idx_end]+" "+keyword+'/place' # 구 + 동 + 헬스장 이름 
    print(naver_map_search_url)

    driver.get(naver_map_search_url)
    time.sleep(12)
    cu = driver.current_url
    res_code = re.findall(r"place/(\d+)", cu) 
    final_url = 'https://pcmap.place.naver.com/place/'+res_code[0]+'/review/visitor#' 
    print(keyword, final_url)
    df['naver_map_url'][i] = res_code[0]
  except IndexError:
    df['naver_map_url'][i] = 'none'
    print('url을 찾지 못했습니당')

이번에 찾을 키워드 :  헤라클레스
https://map.naver.com/v5/search/동작구 흑석동 헤라클레스/place
헤라클레스 https://pcmap.place.naver.com/place/38738534/review/visitor#
이번에 찾을 키워드 :  명문헬스클럽
https://map.naver.com/v5/search/동작구 상도동 명문헬스클럽/place


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


명문헬스클럽 https://pcmap.place.naver.com/place/18704679/review/visitor#
이번에 찾을 키워드 :  에브리데이짐
https://map.naver.com/v5/search/동작구 상도동 에브리데이짐/place
에브리데이짐 https://pcmap.place.naver.com/place/1318853078/review/visitor#
이번에 찾을 키워드 :  국제 헬스
https://map.naver.com/v5/search/동작구 노량진동 국제 헬스/place
국제 헬스 https://pcmap.place.naver.com/place/11831148/review/visitor#
이번에 찾을 키워드 :  엔짐
https://map.naver.com/v5/search/동작구 상도동 엔짐/place
엔짐 https://pcmap.place.naver.com/place/1136266766/review/visitor#
이번에 찾을 키워드 :  마크짐 휘트니스
https://map.naver.com/v5/search/동작구 흑석동 마크짐 휘트니스/place
마크짐 휘트니스 https://pcmap.place.naver.com/place/31564479/review/visitor#
이번에 찾을 키워드 :  피렉스휘트니스센터
https://map.naver.com/v5/search/동작구 상도동 피렉스휘트니스센터/place
피렉스휘트니스센터 https://pcmap.place.naver.com/place/1018025728/review/visitor#
이번에 찾을 키워드 :  프레토 휘트니스
https://map.naver.com/v5/search/동작구 대방동 프레토 휘트니스/place
프레토 휘트니스 https://pcmap.place.naver.com/place/1430563569/review/visitor#
이번에 찾을 키워드 :  이루어짐
https://map.naver.com/v5/search/동작구 노량진동 이루어짐/

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


url을 찾지 못했습니당
이번에 찾을 키워드 :  바디킹PT샵
https://map.naver.com/v5/search/동작구 대방동 바디킹PT샵/place
바디킹PT샵 https://pcmap.place.naver.com/place/1728630366/review/visitor#
이번에 찾을 키워드 :  크로스핏 보라매. 엘루이짐
https://map.naver.com/v5/search/동작구 신대방동 크로스핏 보라매. 엘루이짐/place
크로스핏 보라매. 엘루이짐 https://pcmap.place.naver.com/place/36459268/review/visitor#
이번에 찾을 키워드 :  예스휘트니스센터
https://map.naver.com/v5/search/동작구 상도동 예스휘트니스센터/place
예스휘트니스센터 https://pcmap.place.naver.com/place/20614467/review/visitor#
이번에 찾을 키워드 :  클레어 피트니스 필라테스&PT 이수역점
https://map.naver.com/v5/search/동작구 사당동 클레어 피트니스 필라테스&PT 이수역점/place
클레어 피트니스 필라테스&PT 이수역점 https://pcmap.place.naver.com/place/1099353212/review/visitor#
이번에 찾을 키워드 :  투혼짐
https://map.naver.com/v5/search/동작구 노량진동 투혼짐/place
투혼짐 https://pcmap.place.naver.com/place/35966449/review/visitor#
이번에 찾을 키워드 :  Keep Fit studio
https://map.naver.com/v5/search/동작구 노량진동 Keep Fit studio/place
Keep Fit studio https://pcmap.place.naver.com/place/1621093212/review/visitor#
이번에 찾을 키워드 :  고위드 피트니스
https://m

In [ ]:
df[df['naver_map_url']=="none"]

,id,name,img,grade,lat,lng,phoneNumber,address,naver_map_url
20,20,원투원휘트니스,https://search.pstatic.net/common/?autoRotate=...,4.53,37.512193,126.938498,823-0121,"서울특별시 동작구 장승배기로28길 20, 지1층 (노량진동)",none


# 헬스장 주소 크롤링 

In [ ]:
df['address2'] = ""

In [ ]:
import requests
for i in range(len(df)):
  print(i,"번째 헬스장")
  print(df['name'][i])
  url = 'https://pcmap.place.naver.com/place/'+str(df['naver_map_url'][i])+'/home'
  print(url)
  result = requests.get(url)
  soup = BeautifulSoup(result.content.decode('utf-8','replace'), "html.parser")

  try:
    address = soup.select("._2yqUQ")[0].getText()
    print(address)
    df['address2'][i] = address
  except IndexError:
    continue

0 번째 헬스장
헤라클레스
https://pcmap.place.naver.com/place/38738534/home
서울 동작구 흑석로6가길 21
1 번째 헬스장
명문헬스클럽
https://pcmap.place.naver.com/place/18704679/home


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


서울 동작구 상도로 399 삼호아파트 상가2동 201호
2 번째 헬스장
에브리데이짐
https://pcmap.place.naver.com/place/1318853078/home
서울 동작구 상도로 179 2층
3 번째 헬스장
국제 헬스
https://pcmap.place.naver.com/place/11831148/home
서울 동작구 등용로4길 148
4 번째 헬스장
엔짐
https://pcmap.place.naver.com/place/1136266766/home
서울 동작구 양녕로26길 27 5층
5 번째 헬스장
마크짐 휘트니스
https://pcmap.place.naver.com/place/31564479/home
서울 동작구 서달로 166 덕봉빌딩 5층
6 번째 헬스장
피렉스휘트니스센터
https://pcmap.place.naver.com/place/1018025728/home
서울 동작구 상도로 96 하나은행
7 번째 헬스장
프레토 휘트니스
https://pcmap.place.naver.com/place/1430563569/home
서울 동작구 보라매로 113 코스모빌딩 5층
8 번째 헬스장
이루어짐
https://pcmap.place.naver.com/place/12861635/home
서울 동작구 장승배기로27길 18 지하1층
9 번째 헬스장
바디 디자인샵
https://pcmap.place.naver.com/place/37992177/home
서울 동작구 국사봉1길 12 진선빌딩 3층
10 번째 헬스장
커브스 상도클럽
https://pcmap.place.naver.com/place/34331455/home
서울 동작구 상도로 352 3층
11 번째 헬스장
해가든짐
https://pcmap.place.naver.com/place/37002614/home
서울 동작구 서달로 150 해가든상가 B01호
12 번째 헬스장
코드짐
https://pcmap.place.naver.com/place/36281903/home
서울 동작구 만양로8길 63
13 번

# 주소 -> 위도 경도 

In [ ]:
import requests, json

def get_location(address):
  url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + address
  # 'KaKaoAK '는 그대로 두시고 개인키만 지우고 입력해 주세요.
  # ex) KakaoAK 6af8d4826f0e56c54bc794fa8a294
  headers = {"Authorization": "KakaoAK 07d87cd8a22ca8cdf61219221e33eb78"}
  api_json = json.loads(str(requests.get(url,headers=headers).text))
  address = api_json['documents'][0]['address']
  crd = {"lat": str(address['y']), "lng": str(address['x'])}

  return crd

for i in range(len(df)):
  try: 
    crd = get_location(df['address'][i])
  except:
    try:
      crd = get_location(df['address2'][i])
    except:
      crd = {'lat':0, 'lng':0}
  df['lat'][i], df['lng'][i] = crd['lat'], crd['lng']
  print(df['name'][i], crd)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


헤라클레스 {'lat': '37.5059379074905', 'lng': '126.955856834488'}
명문헬스클럽 {'lat': '37.4938176249105', 'lng': '126.956236455386'}
에브리데이짐 {'lat': '37.504803193551', 'lng': '126.938547350594'}
국제 헬스 {'lat': '37.5119384569884', 'lng': '126.93473418566'}
엔짐 {'lat': '37.4973902075154', 'lng': '126.944257806099'}
마크짐 휘트니스 {'lat': '37.5077644083105', 'lng': '126.962881274043'}
피렉스휘트니스센터 {'lat': '37.4999361621243', 'lng': '126.931383839639'}
프레토 휘트니스 {'lat': '37.4994633095093', 'lng': '126.927896614044'}
이루어짐 {'lat': '37.5116890183844', 'lng': '126.939042544418'}
바디 디자인샵 {'lat': '37.4987849415093', 'lng': '126.929236790898'}
커브스 상도클럽 {'lat': '37.4969894833049', 'lng': '126.952614412075'}
해가든짐 {'lat': '37.5066265781961', 'lng': '126.962183786625'}
코드짐 {'lat': '37.5103107126825', 'lng': '126.9458007938'}
짐퍼스트 동작점 {'lat': '37.4910969139998', 'lng': '126.911813692107'}
엠에이치 토탈 휘트니스 신대방 {'lat': '37.4899432805842', 'lng': '126.907821825149'}
러너 피트니스 {'lat': '37.4970019422426', 'lng': '126.943642572569'}
리턴

In [ ]:
df.loc[df['name']=='클럽컬리넌', 'lat'] = '37.4863323127883'
df.loc[df['name']=='클럽컬리넌', 'lng'] = '126.980927405478'

In [ ]:
df[df['lat']==0]

,id,name,img,grade,lat,lng,phoneNumber,address,naver_map_url,address2


In [ ]:
# idx = df[df['lat']==0].index
# df = df.drop(idx)

In [ ]:
df.to_csv("/content/drive/MyDrive/datasets/헬스장 크롤링/{0}/company_plus.csv".format(region))

In [ ]:
df.head()

,id,name,img,grade,lat,lng,phoneNumber,address,naver_map_url,address2
0,0,헤라클레스,https://search.pstatic.net/common/?autoRotate=...,NaN,37.505938,126.955857,824-8688,서울특별시 동작구 흑석로6가길 21 (흑석동),38738534,서울 동작구 흑석로6가길 21
1,1,명문헬스클럽,https://search.pstatic.net/common/?autoRotate=...,NaN,37.493818,126.956236,814-0579,"서울특별시 동작구 상도로 399, 202호 (상도동, 삼호상가)",18704679,서울 동작구 상도로 399 삼호아파트 상가2동 201호
2,2,에브리데이짐,https://search.pstatic.net/common/?autoRotate=...,4.95,37.504803,126.938547,817-6400,서울특별시 동작구 상도로 179 (상도동),1318853078,서울 동작구 상도로 179 2층
3,3,국제 헬스,NaN,NaN,37.511938,126.934734,02-814-4155,서울특별시 동작구 등용로4길 148 (노량진동),11831148,서울 동작구 등용로4길 148
4,4,엔짐,https://search.pstatic.net/common/?autoRotate=...,4.78,37.49739,126.944258,817-8990,"서울특별시 동작구 양녕로26길 27, 5층 (상도동, 삼익아파트)",1136266766,서울 동작구 양녕로26길 27 5층


In [ ]:
df.tail()

,id,name,img,grade,lat,lng,phoneNumber,address,naver_map_url,address2
46,46,포미짐,NaN,NaN,37.625277,127.080023,02-978-4242,"서울특별시 노원구 공릉로 172, 반야빌딩 5층 (공릉동)",1996116358,서울 노원구 공릉로 172 반야빌딩 5층
47,47,조아Gym,https://search.pstatic.net/common/?autoRotate=...,NaN,37.663455,127.076221,NaN,"서울특별시 노원구 덕릉로 738, 정진145빌딩 2층 (상계동)",1748054991,서울 노원구 덕릉로 738 2층 조아짐
48,48,크로스핏 타키온,https://search.pstatic.net/common/?autoRotate=...,NaN,37.620896,127.058796,NaN,"서울특별시 노원구 광운로 37-1, 태경빌딩 지층 (월계동)",1484852211,서울 노원구 광운로 37-1 지하 1층
49,49,피트니스키 FITNESSKI,https://search.pstatic.net/common/?autoRotate=...,NaN,37.665825,127.066594,NaN,"서울특별시 노원구 한글비석로52길 30, 지층 (상계동)",1642472826,서울 노원구 한글비석로52길 30 지층
50,50,빌리핏 PT 스튜디오,NaN,NaN,37.619613,127.077885,NaN,"서울특별시 노원구 공릉로 105, 성종빌딩 2층 1호 (공릉동)",1104811643,"서울 노원구 공릉로 105 201호(공릉동, 장원빌딩)"
